In [110]:
import numpy  as np
import pandas as pd
import gc
import pprint

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn           as sns

import pandas_alive

from IPython.display import display

leio

In [172]:
df = pd.read_csv("~/Documents/backup_20210417_leio-co/leio_sessions.csv")

In [155]:
df

,Book,Started On,Finished On,Duration,First Page,Last Page,Pages Read,Time/Page
0,Sandman: Edição Especial de 30 Anos – Vol. 9: ...,09/04/21 22:01,09/04/21 23:04,1 h 2 min,1,13,13,4 min 50 s
1,Sandman: Edição Especial de 30 Anos – Vol. 9: ...,17/04/21 16:25,17/04/21 16:55,29 min 34 s,48,76,29,1 min 1 s
2,Sandman: Edição Especial de 30 Anos – Vol. 9: ...,12/04/21 20:11,12/04/21 20:37,26 min 15 s,21,48,28,56 s
3,Sandman: Edição Especial de 30 Anos – Vol. 9: ...,17/04/21 17:29,17/04/21 18:02,32 min 59 s,98,123,26,1 min 16 s
4,Sandman: Edição Especial de 30 Anos – Vol. 9: ...,17/04/21 19:01,17/04/21 19:28,26 min 45 s,123,148,26,1 min 1 s
...,...,...,...,...,...,...,...,...
355,Realismo Capitalista: É mais fácil imaginar o ...,22/12/20 18:32,22/12/20 19:12,39 min 35 s,103,117,15,2 min 38 s
356,Realismo Capitalista: É mais fácil imaginar o ...,22/12/20 17:47,22/12/20 18:17,30 min 3 s,89,103,15,2 min 0 s
357,Realismo Capitalista: É mais fácil imaginar o ...,21/12/20 18:13,21/12/20 18:44,30 min 53 s,55,67,13,2 min 22 s
358,Realismo Capitalista: É mais fácil imaginar o ...,07/12/20 20:48,07/12/20 22:41,42 min 45 s,39,55,17,2 min 30 s


In [157]:
df['dt_start'] = pd.to_datetime(df['Started On'], dayfirst=True).dt.strftime('%Y-%m-%d')
df['dt_end']   = pd.to_datetime(df['Finished On'], dayfirst=True).dt.strftime('%Y-%m-%d')

In [158]:
def duration(_str, unit="s"):
    h,m,s = 0,0,0
    if "h" in _str:
        h = int(_str.split("h")[0])
    if "min" in _str:
        m = int(_str.split("h")[-1].split("min")[0])
    if "s" in _str:
        s = int(_str.split("h")[-1].split("min")[-1].split("s")[0])
    
    if unit == "s":
        return h*3600 + m*60 + s
    if unit == "m":
        return h*60 + m + int(s/60)
    if unit == "h":
        return h + int( (m + int(s/60))/60 )

df['dt_dur_s'] = df['Duration'].apply( lambda x: duration(x, "s") )
df['dt_dur_m'] = df['Duration'].apply( lambda x: duration(x, "m") )
df['dt_dur_h'] = df['Duration'].apply( lambda x: duration(x, "h") )

In [159]:
df_sum = df.sort_values(by="dt_end", ascending=False).groupby(["Book", "dt_end"]).agg( {"Pages Read":'sum'} ).groupby(level=0).cumsum().reset_index()

df_sum

,Book,dt_end,Pages Read
0,A Solidao De Um Quadrinho Sem Fim (Em Portugue...,2021-03-30,161
1,A história do Brasil para quem tem pressa,2020-07-31,19
2,A história do Brasil para quem tem pressa,2020-08-01,42
3,A história do Brasil para quem tem pressa,2020-08-29,60
4,A história do Brasil para quem tem pressa,2020-10-12,103
...,...,...,...
203,Viva o povo brasileiro - Edição especial de 30...,2020-07-10,2
204,Viva o povo brasileiro - Edição especial de 30...,2021-03-26,13
205,Xondaro,2020-11-21,60
206,Índios do Brasil,2020-07-10,32


In [166]:
r = pd.date_range(start = df_sum["dt_end"].min(), end = df_sum["dt_end"].max()).strftime('%Y-%m-%d')
display(pd.DataFrame(index=r).join(df_sum.set_index("dt_end")))
display(pd.DataFrame(index=r).join(df_sum.set_index("dt_end")).fillna(method="ffill"))

,Book,Pages Read
2020-07-10,As Veias Abertas da América Latina,16.0
2020-07-10,Las venas abiertas de América Latina,12.0
2020-07-10,Viva o povo brasileiro - Edição especial de 30...,2.0
2020-07-10,Índios do Brasil,32.0
2020-07-11,NaN,NaN
...,...,...
2021-04-14,NaN,NaN
2021-04-15,NaN,NaN
2021-04-16,NaN,NaN
2021-04-17,"Naoki Urasawa's Monster, Volume 8",434.0


,Book,Pages Read
2020-07-10,As Veias Abertas da América Latina,16.0
2020-07-10,Las venas abiertas de América Latina,12.0
2020-07-10,Viva o povo brasileiro - Edição especial de 30...,2.0
2020-07-10,Índios do Brasil,32.0
2020-07-11,Índios do Brasil,32.0
...,...,...
2021-04-14,Sandman: Edição Especial de 30 Anos – Vol. 9: ...,50.0
2021-04-15,Sandman: Edição Especial de 30 Anos – Vol. 9: ...,50.0
2021-04-16,Sandman: Edição Especial de 30 Anos – Vol. 9: ...,50.0
2021-04-17,"Naoki Urasawa's Monster, Volume 8",434.0


In [175]:
pdf = pd.DataFrame(index=r).join(df_sum.set_index("dt_end")).fillna(method="ffill").reset_index().pivot( 
                index   = 'index',
                columns = 'Book',
                values  = 'Pages Read'
              ).fillna(0)

pdf

Book,A Solidao De Um Quadrinho Sem Fim (Em Portugues do Brasil),A história do Brasil para quem tem pressa,A história dos caxinauás por eles mesmos,Abecedário de personagens do folclore brasileiro,Albert Camus,Angola Janga: Uma História de Palmares,As Veias Abertas da América Latina,Beowulf,Biblioteca Eisner - Um Contrato Com Deus,Bonjour,...,Satsuma Gishiden 2: Crônicas dos Leais Guerreiros de Satsuma,Segredos da comédia stand-up,Sharaz-De: Contos de As Mil e Uma Noites; volume 1 de 2,Sin noticias de Gurb,Tanka,Taz,Tupã tenondé,Viva o povo brasileiro - Edição especial de 30 anos,Xondaro,Índios do Brasil
index,,,,,,,,,,,,,,,,,,,,,
2020-07-10,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,32.0
2020-07-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0
2020-07-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0
2020-07-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0
2020-07-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-04-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-04-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [176]:
pdf.plot_animated(filename   = 'teste3.mp4', 
                  n_visible  = 5,
                  title      = 'TESTE',
                  fixed_max  = True,
                  interpolate_period = False)

Generating BarChartRace, plotting ['A Solidao De Um Quadrinho Sem Fim (Em Portugues do Brasil)', 'A história do Brasil para quem tem pressa', 'A história dos caxinauás por eles mesmos', 'Abecedário de personagens do folclore brasileiro', 'Albert Camus', 'Angola Janga: Uma História de Palmares', 'As Veias Abertas da América Latina', 'Beowulf', 'Biblioteca Eisner - Um Contrato Com Deus', 'Bonjour', 'Cerrado por fútbol', 'Civilizacao Islamica em Trinta Biografias. Os Primeiros Mil Anos (Em Portugues do Brasil)', 'Conto de Areia', 'Couro de Gato: Uma História do Samba', 'Dicionario da Independencia', 'Dilbert. Já Nem Lembro Se Somos Muquiranas ou Espertos', 'Dragão Negro', 'El fútbol a sol y sombra', 'Faces do Humor', 'Historia mínima de Argentina', 'História da vida privada no Brasil - vol. 1', 'Incal (Todo Incal, #1)', 'Introdução a Jacques Lacan', 'Ironias do tempo', 'Kobane Calling: ou como fui parar no meio da guerra na Síria', 'Las venas abiertas de América Latina', 'Macanudo 1', 'Ma

/home/dusodeth/.local/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:238: RuntimeWarning: Glyph 127 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/dusodeth/.local/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:238: RuntimeWarning: Glyph 128 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/dusodeth/.local/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:238: RuntimeWarning: Glyph 129 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/dusodeth/.local/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:238: RuntimeWarning: Glyph 130 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/dusodeth/.local/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:238: RuntimeWarning: Glyph 131 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/dusodeth/.local/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:238: RuntimeWar

BarChartRace(df=Book        A Solidao De Um Quadrinho Sem Fim (Em Portugues do Brasil)  \
index                                                                    
2020-07-10                                                0.0            
2020-07-10                                                0.0            
2020-07-10                                                0.0            
2020-07-10                                                0.0            
2020-07-10                                                0.0            
...                                                       ...            
2021-04-16                                                0.0            
2021-04-16                                                0.0            
2021-04-16                                                0.0            
2021-04-16                                                0.0            
2021-04-17                                                0.0            

Book        A históri

In [121]:
for b in df["Book"].unique():
    print(b)
    subdf = df[df["Book"].isin([b])].sort_values(by="dt_end", ascending=False)
    subg  = subdf.groupby(["dt_end"]).agg( {"Pages Read":'sum'} )
    display(subg.cumsum())

Sandman: Edição Especial de 30 Anos – Vol. 9: Entes Queridos (The Sandman, #9)


,Pages Read
dt_end,
2021-04-09,13
2021-04-12,50
2021-04-17,154


Marighella #LIVRE


,Pages Read
dt_end,
2020-10-11,64


Sandman: Edição Especial De 30 Anos – Vol. 5: Um Jogo de Você (The Sandman, #5)


...

,Pages Read
dt_end,
2020-12-25,43
2020-12-28,57
2020-12-29,92


Sandman: Edição Especial De 30 Anos – Vol. 1: Prelúdios & Noturnos (The Sandman, #1)


,Pages Read
dt_end,
2020-08-09,152
2020-08-10,230
2020-08-11,244


Realismo Capitalista: É mais fácil imaginar o fim do mundo do que o fim do capitalismo?


,Pages Read
dt_end,
2020-11-27,21
2020-12-07,58
2020-12-21,94
2020-12-22,141
2020-12-23,186
2020-12-24,233


In [188]:
pd.DataFrame(index=r).join(df_sum.set_index("dt_end")).fillna(method="ffill")["Pages Read"].cumsum().fillna(method="ffill")

2020-07-10       16.0
2020-07-10       28.0
2020-07-10       30.0
2020-07-10       62.0
2020-07-11       94.0
               ...   
2021-04-14    39754.0
2021-04-15    39804.0
2021-04-16    39854.0
2021-04-17    40288.0
2021-04-17    40442.0
Name: Pages Read, Length: 342, dtype: float64

In [190]:
pd.DataFrame(index=r).join(df_sum.set_index("dt_end")).fillna(method="ffill")["Pages Read"].cumsum().fillna(method="ffill").plot_animated(kind='line',filename="teste.mp4",title="TESTE")

Generating LineChart, plotting ['Pages Read']


/home/dusodeth/.local/lib/python3.8/site-packages/pandas_alive/_base_chart.py:458: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df.columns)
/home/dusodeth/.local/lib/python3.8/site-packages/pandas_alive/_base_chart.py:459: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))


RuntimeError: Ensure that a matplotlib writer library is installed, see https://github.com/JackMcKew/pandas_alive/blob/main/README.md#requirements for more details